In [1]:
#######################################################
##### STRIP TWEET #####################################
#######################################################

#import pandas as pd
#df = pd.read_csv("train2017.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
#corpus = df[3].tolist()

import nltk
import string
import re

f = open("small.tsv")
line = f.readline()

tweets = []
translate_table = dict((ord(char), None) for char in string.punctuation)   
cnt = 1
while line:
    pretext = ' '.join(line.split()[3:])
    pretext = ' '.join(word for word in pretext.split(' ') if not word.startswith('@')) #remove @user
    pretext = re.sub(r"http\S+", "", pretext) #remove link
    pretext = pretext.translate(translate_table) #remove symbols 
    #remove numbers??
    
    tweet = pretext.lower()
    tweets.append(tweet)
    line = f.readline()
    cnt += 1

#print(tweets)

In [2]:
#######################################################
##### TOKENIZATION ####################################
#######################################################

from nltk import word_tokenize

tokens = []
for tweet in tweets:
    token = []
    token = word_tokenize(tweet)
    tokens.append(token)

print(tokens)

[['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'going', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], ['theo', 'walcott', 'is', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], ['its', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], ['iranian', 'general', 'says', 'israels', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missiles', 'keep', 'talking', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'finding', 'out'], ['tehran', 'mon', 'amour', 'obama', 'tried', 'to', 'establish', 'ties', 'with', 'the', 'mullahs', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], ['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], ['mashed', 'out', 'to', 'niggas', 'in', 'paris', 'in', 'the', 'club', 'while', 'in', 'paris', 'as', 'cliche', 'as', 'it', 'may', 'sound', 'weouthere'], ['larry', 'bird', 'is', 'ra

In [3]:
from nltk import pos_tag

from nltk.corpus import wordnet
from nltk.stem import  WordNetLemmatizer
# Lemmatize with POS Tag

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

tweets = []
for token_list in tokens:
    lemmatized = []
    for word in token_list:
        lemmatized.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
    tweets.append(lemmatized)

#Rprint(tweets)

In [4]:
corpus = []
for tweet in tweets:
    corpus.append(" ".join(str(word) for word in tweet))

print(corpus)

['gas by my house hit 339 im go to chapel hill on sat 😂', 'theo walcott be still shit fan watch rafa and johnny deal with him on saturday', 'it not that im a gsp fan i just hate nick diaz cant wait for february', 'iranian general say israel iron dome cant deal with their missile keep talk like that and we may end up find out', 'tehran mon amour obama try to establish tie with the mullah via no barack obama vote mitt romney', 'i sat through this whole movie just for harry and ron at christmas ohlawd 911', 'mash out to nigga in paris in the club while in paris a cliche a it may sound weouthere', 'larry bird be ranked 4th alltime not include lebron or kobe just sayin']


In [5]:
#BAG-OF-WORDS VECTORIZATION
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_BOW = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())
#print(X.toarray()[0])
#print(X_BOW.toarray())


In [6]:
#TF-IDF VECTORIZATION
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_TFIDF = vectorizer.fit_transform(corpus)
#print(X_TFIDF.toarray())


In [7]:
#we have 3 options: word2vec, word2vec in sklearn, doc2vec
from gensim.test.utils import common_texts

#IMPORT WORD2VEC
from gensim.models import Word2Vec
nested_corpus = [corpus[i:i+1] for i in range(0, len(corpus), 1)]
model = Word2Vec(tokens, size=100, window=5, min_count=1, workers=4) #size of vector is 100
model.train(tokens, total_examples=model.corpus_count,epochs=model.epochs)  # train word vectors
print(model['gas'])

#IMPORT WORD2VEC LIBRARY AVAILABLE IN SKLEARN

#from gensim.sklearn_api import W2VTransformer
#total = sum(tokens,[])
#vectorizer = W2VTransformer(size=10, min_count=1, seed=1)
#print(corpus)
#X_embeddings = vectorizer.fit(total).transform(tokens)



[-0.00285832  0.00169726  0.00501415 -0.00082891 -0.00054391 -0.00339097
  0.00425513 -0.00298445 -0.00040991 -0.00499388 -0.00082528  0.0025047
 -0.00151833 -0.00124463  0.00045757 -0.00356886 -0.00128379  0.00010001
  0.00477899 -0.00155952  0.00283658 -0.00495182  0.00040114  0.00066271
  0.00106843  0.0048173  -0.00476683  0.00379649 -0.00034901  0.00477842
 -0.00232284  0.004271   -0.00127936  0.00196828 -0.00162923 -0.00096433
  0.00247572  0.00084297 -0.00033414 -0.00493055  0.00315856  0.00163823
 -0.00342612 -0.00133807 -0.00200222  0.00410518 -0.00045238  0.00057336
  0.00429248  0.0032558  -0.00393762 -0.00308446  0.00021209  0.00419186
 -0.00373008  0.00040428  0.00145865  0.00032684  0.00423606  0.00022076
 -0.00374816  0.00028931 -0.00199852  0.00309567  0.00149621 -0.00254338
 -0.00043051  0.00423646  0.00227635 -0.00249813  0.00123682 -0.00210515
  0.00483974  0.00187356  0.00150603  0.00475347  0.00177318  0.00285438
  0.00090927 -0.00070616 -0.00070972  0.00383707  0.

/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [8]:
#Doc2vec is equal with word2vec but is more appropriate for phrases (vectorize phrases instead of words). I.e 
#1.Manos leaves the office every day at 18:00 to catch his train
#2. This season is called Fall, because leaves fall from the trees.
#In this way we can capture the difference between the same word used in a different context. For example we now have a
#different representation of the word “leaves” in the above two sentences

#IMPORTANT: parameteres in my case is random, we have to pay attention to select the right ones

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(corpus)]
print(tagged_data)
model = Doc2Vec(size=200,
                alpha=model.alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

max_epoch = 20
for epoch in range(max_epoch):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
X_embeddings_array = model.docvecs
#convert array to list
X_embeddings=[]
for i in range(len(X_embeddings_array)):
    X_embeddings.append(X_embeddings_array[i].tolist())

/home/dimitra/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


[TaggedDocument(words=['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'go', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], tags=['0']), TaggedDocument(words=['theo', 'walcott', 'be', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], tags=['1']), TaggedDocument(words=['it', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], tags=['2']), TaggedDocument(words=['iranian', 'general', 'say', 'israel', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missile', 'keep', 'talk', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'find', 'out'], tags=['3']), TaggedDocument(words=['tehran', 'mon', 'amour', 'obama', 'try', 'to', 'establish', 'tie', 'with', 'the', 'mullah', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], tags=['4']), TaggedDocument(words=['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], tags=

In [9]:
#Add characteristics to embeddings
import pandas as pd
import csv
lexica =[pd.read_csv("lexica/affin/affin.txt", sep='\t', header=None),
              pd.read_csv("lexica/emotweet/valence_tweet.txt", sep='\t', header=None),
              pd.read_csv("lexica/generic/generic.txt", sep='\t', engine="python" ,quoting=csv.QUOTE_NONE,header=None),
              pd.read_csv("lexica/nrc/val.txt", sep='\t', engine="python",quoting=csv.QUOTE_NONE, header=None),
              pd.read_csv("lexica/nrctag/val.txt", sep='\t', header=None)]

#mean valence of each tweet
for tweet in range(len(tokens)):   #for each tweet
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(tokens[tweet])].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(tokens[tweet])][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:
             X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector

#length of each tweet               
for tweet in range(len(tokens)):   #for each tweet
    X_embeddings[tweet].append(len(tokens[tweet]))


    
#min and max valence of each tweet
for tweet in range(len(tokens)):   #for each tweet
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(tokens[tweet])].empty: #search for tweets' tokens in lexicon
            max_valence = df[df[0].isin(tokens[tweet])][1].max() #get average sentiment for this tweet
            min_valence = df[df[0].isin(tokens[tweet])][1].min()
            X_embeddings[tweet].append(max_valence)
            X_embeddings[tweet].append(min_valence)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector

    
#mean valence for each half of a tweet
for tweet in range(len(tokens)):   #for each tweet
    entire_tweet = tokens[tweet]
    first_half= entire_tweet[:len(entire_tweet)//2]
    second_half= entire_tweet[len(entire_tweet)//2:]
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(first_half)].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(first_half)][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
        if not df[df[0].isin(second_half)].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(second_half)][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
        
    
            
for i in range(len(X_embeddings)):
    print(len(X_embeddings[i]))

#sentiment_sum = 0
#words_found = 0
#for j in range(len(tokens[i])):
#    if not df[df[0] == tokens[i][j]].empty: #search for tweets' tokens in lexicon
#        sentiment_sum += df[df[0] == tokens[i][j]].iloc[0][1] #get sentiment
#        words_found++
        

        


226
226
226
226
226
226
226
226


In [20]:
####################
#CONSTRUCT Y_LABELS#
####################

import pandas as pd
df = pd.read_csv("small.tsv", sep='\t', header=None)
y_labels = df[2].tolist() #sentiments


for n, value in enumerate(y_labels):
    if value == "positive":
        y_labels[n] = 2
    elif value =="negative":
        y_labels[n] = 0
    else:
        y_labels[n] =1



[2, 0, 0, 0, 1, 1, 1, 1]
